# Assembling

This notebook is used to assemble a processable h5ad object for subsequent notebooks.

## Fill in input data, output and settings

In [1]:
####################### TEST NAME ###################################
test = 'Aorta'
################### DEFINING STRATEGY ###############################
from_h5ad = True #option 1
assembling_10_velocity = False #option 2
assembling_10_public = False #option 3
convert_seurat = False #option 4

###################### INPUT DATA ###################################

#For option 1: The path to an existing .h5ad file
h5ad_path = '/home/rstudio/data/anndata/cropped_146.h5ad'
#################### Calc QC Columns ###############################
n_threads = 8

fragments_file = '/home/rstudio/data/bamfiles/fragments_cropped_146.bed'
bam_file = '/home/rstudio/data/bamfiles/sorted_cropped_146.bam'
#promoters_gtf =  '/mnt/flatfiles/organisms/new_organism/homo_sapiens/104/homo_sapiens.104.promoters2000.gtf'
promoters_gtf = '/home/rstudio/data/homo_sapiens.104.promoters2000.gtf'


##################### OUTPUT DATA ###################################
output_dir = '/home/rstudio/processed_data'

## Import modules

In [2]:
# sctoolbox modules 
import sctoolbox.atac_tree as sub_tree
import sctoolbox.creators as cr
import sctoolbox.fragment_length as fragments
import sctoolbox.calc_overlap_pct as overlap
# import episcanpy
import episcanpy as epi

## Setup path handling object 

In [3]:
# make an instance of the class
tree = sub_tree.ATAC_tree()
# set processing/output directory
tree.processing_dir = output_dir
# set sample/experiment.. 
tree.run = test

## Read in data

### Option 1: Read from .h5ad

In [4]:
if from_h5ad:
    
    adata = epi.read_h5ad(h5ad_path)
    
    #Add information to the infoprocess
    cr.build_infor(adata, "Input_for_assembling", h5ad_path)
    cr.build_infor(adata, "Strategy", "Read from h5ad")

## Inspect adata

In [5]:
display(adata)

AnnData object with n_obs × n_vars = 373 × 3830
    obs: 'barcode', 'TN', 'UM', 'PP', 'UQ', 'CM', 'file', 'sample'
    var: 'name'
    uns: 'infoprocess', 'color_set'

In [6]:
display(adata.var)

,name
0,b'chr1':9993-10421
1,b'chr1':29089-29368
2,b'chr1':180548-180799
3,b'chr1':199689-199928
4,b'chr1':629307-629902
...,...
3825,b'chrY':318869-319190
3826,b'chrY':386933-387197
3827,b'chrY':1453156-1453412
3828,b'chrY':1591522-1591761


In [7]:
display(adata.obs)

,barcode,TN,UM,PP,UQ,CM,file,sample
1,AAACTACCAGAAACCCGAGATA,33.0,18.0,18.0,15.0,0.0,/home/rstudio/data/snap/cropped_146.snap,MB
2,AAACTACCAGAAACCTAAGTGG,52.0,36.0,35.0,32.0,1.0,/home/rstudio/data/snap/cropped_146.snap,MB
3,AAACTACCAGAAACGGATCAGT,27.0,19.0,19.0,19.0,0.0,/home/rstudio/data/snap/cropped_146.snap,MB
4,AAACTACCAGAAACGTCCCGTT,6223.0,5231.0,5213.0,3779.0,18.0,/home/rstudio/data/snap/cropped_146.snap,MB
5,AAACTACCAGAAACTAGCCCTA,41.0,29.0,29.0,26.0,0.0,/home/rstudio/data/snap/cropped_146.snap,MB
...,...,...,...,...,...,...,...,...
369,AAACTACCAGCTTGGCAACAGC,26.0,20.0,20.0,20.0,1.0,/home/rstudio/data/snap/cropped_146.snap,MB
370,AAACTACCAGCTTTAACTGCGC,29.0,20.0,20.0,19.0,0.0,/home/rstudio/data/snap/cropped_146.snap,MB
371,AAACTACCAGCTTTAGCCCTAT,32.0,24.0,24.0,24.0,1.0,/home/rstudio/data/snap/cropped_146.snap,MB
372,AAACTACCAGCTTTAGCTGACT,10437.0,9231.0,9206.0,6209.0,17.0,/home/rstudio/data/snap/cropped_146.snap,MB


## Check for QC related columns

### 1. Check if mean fragment length column exists


In [ ]:
mfl_column = fragments.check_mfl(adata)
mfl_column

### 2. Check if percentage of reads in promoters column exists

In [8]:
pct_rip_column = overlap.check_pct_fragments_in_promoters(adata)
pct_rip_column

False

## Calculate missing columns

### 1. calc mean fragment length if missing 

In [9]:
if not mfl_column:
    adata = fragments.add_mfl_fragment(fragments_file, adata, n_threads)
adata.obs

NameError: name 'mfl_column' is not defined

### 2. Promotor enrichment 

In [10]:
adata.obs = adata.obs.set_index('barcode')

In [12]:
if not pct_rip_column:
    overlap.pct_fragments_in_promoters(adata, promoters_gtf, bam_file=bam_file, cb_col=None, nproc=1)
adata.obs

Converting BAM to fragments file! This may take a while...
Finished creating fragments file. Now sorting...
Finished sorting fragments
Converting GTF to sorted BED...
Finding overlaps...
Calculating percentage...
Adding results to adata object...
Done


,TN,UM,PP,UQ,CM,file,sample,n_total_fragments,n_fragments_in_promoters,pct_fragments_in_promoters
barcode,,,,,,,,,,
AAACTACCAGAAACCCGAGATA,33.0,18.0,18.0,15.0,0.0,/home/rstudio/data/snap/cropped_146.snap,MB,17,5.0,0.294118
AAACTACCAGAAACCTAAGTGG,52.0,36.0,35.0,32.0,1.0,/home/rstudio/data/snap/cropped_146.snap,MB,37,10.0,0.270270
AAACTACCAGAAACGGATCAGT,27.0,19.0,19.0,19.0,0.0,/home/rstudio/data/snap/cropped_146.snap,MB,20,7.0,0.350000
AAACTACCAGAAACGTCCCGTT,6223.0,5231.0,5213.0,3779.0,18.0,/home/rstudio/data/snap/cropped_146.snap,MB,5118,1221.0,0.238570
AAACTACCAGAAACTAGCCCTA,41.0,29.0,29.0,26.0,0.0,/home/rstudio/data/snap/cropped_146.snap,MB,28,6.0,0.214286
...,...,...,...,...,...,...,...,...,...,...
AAACTACCAGCTTGGCAACAGC,26.0,20.0,20.0,20.0,1.0,/home/rstudio/data/snap/cropped_146.snap,MB,17,7.0,0.411765
AAACTACCAGCTTTAACTGCGC,29.0,20.0,20.0,19.0,0.0,/home/rstudio/data/snap/cropped_146.snap,MB,15,5.0,0.333333
AAACTACCAGCTTTAGCCCTAT,32.0,24.0,24.0,24.0,1.0,/home/rstudio/data/snap/cropped_146.snap,MB,19,4.0,0.210526


## Inspect adata.obs

In [ ]:
adata.obs

## Save adata to .h5ad

In [ ]:
adata_output = tree.assembled_anndata
adata_output

In [ ]:
#Saving the data
cr.build_infor(adata, "Test_number", test)
cr.build_infor(adata, "Anndata_path", output_dir)

adata_output = tree.assembled_anndata
adata.write(filename=adata_output)